In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
import cv2 as cv
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Load the saved model
model = torch.load('my_model.pth', map_location=device)
# Define the transform to apply to the image
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor()
])

In [ ]:
ss = cv.ximgproc.segmentation.createSelectiveSearchSegmentation()
model.eval()

path = "airplanes/test/images/"

img = cv.imread(os.path.join(path, 'airplane_0010.jpg'))
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
ssresults = ss.process()
imout = img.copy()
for e, result in enumerate(ssresults):
    if e < 2000:
        x, y, w, h = result
        timage = imout[y:y+h, x:x+w]
        pil_image = Image.fromarray(cv.cvtColor(timage, cv.COLOR_BGR2RGB))
        img_tensor = transform(pil_image)
        img_tensor = img_tensor.unsqueeze(0).to(device)
        with torch.no_grad():
            out = model(img_tensor)
            print(out)
        if out[0][0] > 0.9:
            cv.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 0), 1, cv.LINE_AA)
plt.figure()
plt.imshow(imout)
plt.show()